<a href="https://colab.research.google.com/github/peeush-the-developer/LGMVIP-DataScience/blob/main/Handwritten-Math-Equation-Solver/1_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten Math Equation solver

We can approach this problem statement in the following steps:

1. We are given a dataset of images for different Math symbols using which we can train our Image classification model to classify images as math symbols.
2. Now, we take an image with complete math equation and try to segregate each symbol into smaller images
3. Pass these smaller images to find the actual symbol and then solve the equation as a normal math equation.

It looks easy with the problem divided into above steps. 

Let's get started!

## Train a CNN model to classify given image into Math symbol

1. Download the dataset with math symbol images
2. Train a CNN model to classify them into Math symbol
3. Evaluate the model

### 1. Download the dataset with math symbol images

[Dataset link]()